#### Desmembrar todas as páginas de um arquivo
    pyphf2 e depois instalar 

In [11]:
import PyPDF2 as pyf
from pathlib import Path
arquivo_pdf = pyf.PdfReader('MGLU_ER_3T20_POR.pdf')

for i, pagina in enumerate(arquivo_pdf.pages):
    novo_pdf = pyf.PdfWriter()
    novo_pdf.add_page(pagina)
    with Path(f'paginas/arquivo {i+1} pagina.pdf').open('wb') as arquivo:
        novo_pdf.write(arquivo)

- Juntar PDFs


In [22]:
import PyPDF2 as pyf
from pathlib import Path

num_paginas = [1, 16, 14]
novo_arquivo = pyf.PdfWriter()
for num in num_paginas:
    pagina_pdf = pyf.PdfReader(f'paginas/arquivo {num} pagina.pdf')
    novo_arquivo.add_page(pagina_pdf.pages[0])

with Path(f'consolidade.pdf').open('wb') as arquivo:
    novo_arquivo.write(arquivo)

- Mesclar PDFs

In [21]:
import PyPDF2 as pyf
from pathlib import Path

pdf_mesclado = pyf.PdfMerger()
pdf_mesclado.append('MGLU_ER_3T20_POR.pdf')
pdf_mesclado.append('MGLU_ER_4T20_POR.pdf')

with Path(f'Mesclado.pdf').open('wb') as arquivo:
    pdf_mesclado.write(arquivo)

- Inserindo uma página dentro do PDF

In [25]:
import PyPDF2 as pyf
from pathlib import Path

pdf_mesclado = pyf.PdfMerger()
pdf_mesclado.append('MGLU_ER_3T20_POR.pdf')
pdf_mesclado.merge(1, 'paginas/arquivo 1 pagina.pdf')

with Path(f'relatório 2 Trimestre.pdf').open('wb') as arquivo:
    pdf_mesclado.write(arquivo)

-   Rodar página

In [28]:
arquivo_pdf_original = pyf.PdfReader('MGLU_ER_3T20_POR.pdf')

novo_arquivo = pyf.PdfWriter()

for pagina in arquivo_pdf_original.pages:
    pagina.rotate(180)
    novo_arquivo.add_page(pagina)

with Path(f'paginas Rotacionais.pdf').open('wb') as arquivo:
    novo_arquivo.write(arquivo)

-   Pegar metadados do PDF

In [31]:
arquivo = pyf.PdfReader('MGLU_ER_3T20_POR.pdf')

metadados_arquivo = arquivo.metadata
print(metadados_arquivo)

{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}


-   Obter texto específicos! E verificar em quais páginas ele se encontra

In [50]:
texto_referencia = '| Despesas com Vendas'

for i, pagina in enumerate(arquivo.pages):
    texto_pagina = pagina.extract_text()
    if texto_referencia in texto_pagina:
        print('Numero de página: ', i+1)
        texto_analisar = texto_pagina

Numero de página:  10


-   Obter um texto fazendo uma análise dentro do texto. No caso |

In [53]:
posicao_inicial = texto_analisar.find(texto_referencia)
posicao_final = texto_analisar.find('|', posicao_inicial + 1)

texto_final = texto_analisar[posicao_inicial:posicao_final]

print(texto_final)

| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço,  especialmente em  atendimento e logística.  
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus  os 
9M19).  
 



-   Trabalhando com tabelas em PDFs, usando biblioteca tabula

In [69]:
import tabula

tabelas = tabula.read_pdf('MGLU_ER_3T20_POR.pdf', pages=5)
df_resultado = tabelas[0]
##Excluir linhas totalmente vazias
df_resultado = df_resultado.dropna(how='all', axis=1)

##Excluir colunas totalmente vazias
df_resultado = df_resultado.dropna(how='all', axis=0)

df_resultado.columns = df_resultado.iloc[0]
df_resultado = df_resultado.iloc[1:]

#Deixar os número em sequ~encia
df_resultado = df_resultado.reset_index(drop=True)


display(df_resultado)

,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
0,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
1,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
2,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
3,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
4,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
5,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
6,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
7,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
8,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"
9,Lucro Bruto - Ajustado,"2.178,7","1.488,9","46,3","5.034,","3.964,6","27,0"


-   Ler mais de uma tabela de um PDF

In [84]:
import tabula

tabelas = tabula.read_pdf('MGLU_ER_3T20_POR.pdf', pages=12)

for tabela in tabelas:
    tabela = tabela.dropna(how='all', axis=0)
    tabela = tabela.reset_index(drop=True)


### Apenas a primeira tabela está com cabeçalho errado, vamos separar ela para concertar
df_capitalgiro = tabelas[0]
tabela = tabela.dropna(how='all', axis=0)
tabela = tabela.reset_index(drop=True)
### Usando outro processo para achar o cabeçalho
tabelas2 = tabula.read_pdf('MGLU_ER_3T20_POR.pdf', pages=12, lattice=True)
capitalgiro2 = tabelas2[0]
capitalgiro2 = capitalgiro2.dropna(how='all', axis=0)
capitalgiro2 = capitalgiro2.dropna(how='all', axis=1)
capitalgiro2 = capitalgiro2.reset_index(drop=True)
colunas = capitalgiro2.iloc[0]
colunas = colunas.dropna()
df_capitalgiro.columns = colunas
display(df_capitalgiro)

,R$ milhões,Dif 12U,set-2,jun-2,mar-2,dez-1,set-1
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"


### biblioteca pikepdf
-   captar imagens

In [93]:
from pikepdf import Pdf, PdfImage

arquivo = Pdf.open('MGLU_ER_3T20_POR.pdf')

for pagina in arquivo.pages:
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imagens/{nome}')